In [5]:
import keras
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras import Model, layers
from keras import Input

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import os
import pickle as pk

import numpy as np


In [12]:
# get data or pickle files if there

imdb_dir = '../../../data/aclImdb/test'
# train_dir = os.path.join(imdb_dir, 'train')
labels = list()
texts = list()

# Processing the labels of the raw IMDB data
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(imdb_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1757: character maps to <undefined>

In [ ]:
# Tokenizing the data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.optimizers import RMSprop


In [ ]:
# Tokenizing the data

# cut off reviews after 500 words
max_len = 500 
# train on 10000 samples
training_samples = 10000
 # validate on 10000 samples 
validation_samples = 10000
# consider only the top 10000 words
max_words = 10000 

tokenizer_path = 'tokenizer'
# import tokenizer with the consideration for only the top 500 words
tokenizer = Tokenizer(num_words=max_words) 
# fit the tokenizer on the texts
tokenizer.fit_on_texts(texts) 
# convert the texts to sequences
sequences = tokenizer.texts_to_sequences(texts) 

# save the tokenizer
with open(os.path.join(tokenizer_path, 'tokenizer_m1.pickle'), 'wb') as handle:
    pk.dump(tokenizer, handle, protocol=pk.HIGHEST_PROTOCOL)

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens. ' % len(word_index))

 # pad the sequence to the required length to ensure uniformity
data = pad_sequences(sequences, maxlen=max_len)
print('Data Shape: {}'.format(data.shape))

labels = np.asarray(labels)
print("Shape of data tensor: ", data.shape)
print("Shape of label tensor: ", labels.shape)

In [ ]:
# split the data into training and validation set but before that shuffle it first
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:training_samples + validation_samples]
y_val = labels[training_samples:training_samples + validation_samples]

# test_data
x_test = data[training_samples+validation_samples:]
y_test = labels[training_samples+validation_samples:]

In [ ]:
# model definition

callback_list = [
    keras.callbacks.EarlyStopping(
        patience=1,
        monitor='acc',
    ),
    
    keras.callbacks.TensorBoard(
        log_dir='log_dir_m1',
        histogram_freq=1,
        embeddings_freq=1,
    ),

    keras.callbacks.ModelCheckpoint(
        monitor='val_loss',
        save_best_only=True,
        filepath='model/movie_sentiment_m1.h5',
    ),

    keras.callbacks.ReduceLROnPlateau(
        patience=1,
        factor=0.1,
    )
]

# layer developing
text_input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(max_words, 50)(text_input_layer)
text_layer = Conv1D(256, 3, activation='relu')(embedding_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = Conv1D(256, 3, activation='relu')(text_layer)
text_layer = MaxPooling1D(3)(text_layer)
text_layer = GlobalMaxPooling1D()(text_layer)
text_layer = Dense(256, activation='relu')(text_layer)
output_layer = Dense(1, activation='sigmoid')(text_layer)
model = Model(text_input_layer, output_layer)
model.summary()
model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])


In [ ]:
# multi-input test
history = model.fit(x_train, y_train, epochs=50, batch_size=128, callbacks=callback_list,
                    validation_data=(x_val, y_val))

In [ ]:
# plotting the results
acc = history.history.get('acc')
val_acc = history.history.get('val_acc')
loss = history.history.get('loss')
val_loss = history.history.get('val_loss')

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()


In [4]:
input_array = np.random.randint(1000, size=(32, 10))
input_array

array([[235, 671, 692, 954, 104, 328,  58,   5, 927,   4],
       [422, 771, 353, 862, 858, 993, 544,  10, 836, 456],
       [889, 478, 535, 616, 101, 234, 915, 803, 920, 909],
       [547,  16, 367, 548, 843, 926,  89, 666, 769,  65],
       [408, 905, 632, 927, 137, 237, 206, 509, 484,  78],
       [373, 607, 963, 924, 179, 806, 861, 815,  36, 867],
       [252, 529, 824, 704, 821, 433, 457, 753, 200, 920],
       [387, 348, 479, 495, 634, 569, 907, 149, 891, 315],
       [ 94, 212, 601, 445, 172, 279, 380, 228, 376, 738],
       [759, 569, 592, 354, 610,  50, 737, 418, 695, 567],
       [404, 119, 452, 281, 204, 559, 697,  31, 399, 313],
       [ 86, 980,  73, 548, 974, 183,  22,  39, 172, 760],
       [172, 646, 824, 252,  40, 671, 300, 871, 302, 990],
       [459,  81, 803, 365, 588,  60, 223, 377, 214, 754],
       [841, 840, 107, 825, 604, 366, 863, 126,   0, 237],
       [699, 905, 914, 792, 950, 667, 381, 765, 562, 907],
       [437, 959, 157, 763, 994, 670, 262, 902, 501, 824

In [6]:
import tensorflow as tf

category_indices = [0, 1, 2, 2, 1, 0]
unique_category_count = 3
inputs = tf.one_hot(category_indices, unique_category_count)
print(inputs.numpy())

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [11]:
from keras.preprocessing.text import one_hot
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[38, 19], [40, 11], [29, 45], [20, 11], [12], [10], [23, 45], [33, 40], [23, 11], [35, 39, 19, 38]]
